In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import optuna
import japanize_matplotlib

In [ ]:
# 使用するデータ
suumo = pd.read_csv('./data/csv/SuumoTokyoShaped20221026.csv')

In [7]:
# 変数全部
feature_name = [
    'カテゴリ',
    '建物名',  # いらない
    '経度',  # いる？
    '緯度',  # いる？
    '都道府県',  # いらない
    '市区町村',
    '市区町村以下',  # いらない
    '皇居からの距離',
    '経度(区)',  # いらない
    '緯度(区)',  # いらない
    '路線',
    '駅',
    '経度(駅)',  # いらない
    '緯度(駅)',  # いらない
    '歩',
    'バス',
    '車',
    '最寄駅からの距離',
    '築年数',
    '地上',
    '地下',
    '階数',
    '階',
    '賃料',
    '管理費',  # いる？
    '敷金',  # いる？
    '礼金',  # いる？
    '間取り',
    '部屋数', 
    '間取り_S', 
    '間取り_L', 
    '間取り_D', 
    '間取り_K',
    '専有面積',
    'url',  # いらない
]

In [ ]:
# 変数（絶対使わないであろう変数を削除）
feature_name = [
    'カテゴリ',
    '経度',  # いる？
    '緯度',  # いる？
    '市区町村',
    '皇居からの距離',
    '路線',
    '駅',
    '歩',
    'バス',
    '車',
    '最寄駅からの距離',
    '築年数',
    '地上',
    '地下',
    '階数',
    '階',
    '賃料',
    '管理費',  # いる？
    '敷金',  # いる？
    '礼金',  # いる？
    '間取り',
    '部屋数', 
    '間取り_S', 
    '間取り_L', 
    '間取り_D', 
    '間取り_K',
    '専有面積',
]

In [ ]:
# 変数（だいたい使うであろう皆さん）
feature_name = [
    'カテゴリ',
    '市区町村',
    '皇居からの距離',
    '路線',
    '駅',
    '歩',
    'バス',
    '車',
    '最寄駅からの距離',
    '築年数',
    '地上',
    '地下',
    '階数',
    '階',
    '賃料',
    '間取り',
    '部屋数', 
    '間取り_S', 
    '間取り_L', 
    '間取り_D', 
    '間取り_K',
    '専有面積',
]

# とりあえず何も考えずLightGBM

結論：

In [ ]:
# 必要ならデータ加工(未知データに対して実行できないものはやらない)
# シャッフルしてtrainとtestに分ける
# 調整が必要ならtrainをさらにtrainとvalidに分ける
# 必要ならデータ加工
# モデル構築
# モデル学習
# 学習を評価

In [ ]:
# 準備フェイズ

# 事前のデータ加工なし
suumo_light = copy.copy(suumo.sample(frac=1,random_state=2))

# 変数は(だいたい使うであろう皆さんを)全部そのまま使用
features = [
    'カテゴリ',
    '市区町村',
    '皇居からの距離',
    '路線',
    '駅',
    '歩',
    'バス',
    '車',
    '最寄駅からの距離',
    '築年数',
    '地上',
    '地下',
    '階数',
    '階',
    '間取り',
    '部屋数', 
    '間取り_S', 
    '間取り_L', 
    '間取り_D', 
    '間取り_K',
    '専有面積',
]
target_feature = '賃料'

In [ ]:
# Xとyを定義
X = suumo_light[features]
y = suumo_light[target_feature]


# 学習用とテスト用に分割
train_test = 0.1
train_valid = 0.8

X_train_before, X_test, y_train_before, y_test = train_test_split(X, y, train_size = train_test, random_state = 0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_before, y_train_before, train_size = train_valid, random_state = 0)

In [ ]:
# きれいに分かれているかチェックシステム

i = 0
max_ = suumo,iloc[:,i].max()
min_ = suumo,iloc[:,i].min()

fig = plt.figure()
ax1 = fig.add_subplot(1,3,1)
ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,3,3)

ax1.hist(X_train.iloc[:,i], bins=50)
plt.title('train')
ax2.hist(X_valid.iloc[:,i], bins=50)
plt.title('valid')
ax3.hist(X_test.iloc[:,i], bins=50)
plt.title('test')

plt.xlim(min_,max_)
plt.show()

In [ ]:
# きれいに分かれているかチェックシステム

max_ = suumo,iloc[:,i].max()
min_ = suumo,iloc[:,i].min()

fig = plt.figure()
ax1 = fig.add_subplot(1,3,1)
ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,3,3)

ax1.hist(y_train, bins=50)
plt.title('train')
ax2.hist(y_valid, bins=50)
plt.title('valid')
ax3.hist(y_test, bins=50)
plt.title('test')

plt.xlim(min_,max_)
plt.show()

In [ ]:
# モデルを構築、学習

LGBM = lgb.LGBMRegressor(random_state = 0)

verbose_eval = 1
LGBM.fit(X_train,y_train,
        eval_set=[(X_valid,y_valid)],
        eval_metrics = 'mse',
        callbacks = [lgb.early_stopping(stopping_rounds=10,verbose=True),
                    lgb.log_evaluation(verbose_eval)])

In [10]:
# それぞれの予測結果を収納
y_train_pred = LGBM.predict(X_train)
y_valid_pred = LGBM.predict(X_valis)
y_test_pred = LGBM.predict(X_test)

In [ ]:
# それぞれのr2スコアを算出
print('train:',r2_score(y_train,y_train_pred))
print('train:',r2_score(y_valid,y_valid_pred))
print('train:',r2_score(y_test,y_test_pred))

In [ ]:
# それぞれのMSEを算出
print('train:',mean_squared_error(y_train,y_train_pred))
print('train:',mean_squared_error(y_valid,y_valid_pred))
print('train:',mean_squared_error(y_test,y_test_pred))

In [ ]:
# それぞれの残差プロット
fig = plt.figure()
ax1 = fig.add_subplot(1,3,1)
ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,3,3)

ax1.scatter(y_train_pred, (y_train_pred - y_train))
ax1.title('train')
ax2.scatter(y_train_pred, (y_train_pred - y_train))
ax2.title('valid')
ax3.scatter(y_train_pred, (y_train_pred - y_train))
ax3.title('test')

plt.show()

In [ ]:
# 変数重要度
pd.DataFrame(np.c_[features, LGBM.feature_importances_])

In [ ]:
# optuna を使いたい場合はこちら

kf = KFold(n_split=5)
def objective(trial):
    n_extimators = 1000
    num_leaves = trial.suggest_int('num_leaves',2,64)
    min_child_samples = trial.suggest_int('min_child_samples', 1,10)
    learning_rate = trial.suggest_float('learning_rate',0.001,1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree',0,1)
    subsample = trial.suggest_float('subsample',0,1)
    reg_alpha = trial.suggest_float('reg_alpha',0,1)
    
    LGBM = lgb.LightGBMRegressor(
        n_estimators = n_estimators,
        num_leaves = num_leaves,
        min_child_samples = min_child_samples,
        learning_rate = learning_rate,
        colsamples_bytree = colsamples_bytree,
        subsample = subsample,
        reg_aplha = reg_alpha
    )
    
    result = cross_validate(
        estimator = LGBM,
        X = X_trian_before,
        y = y_train_before,
        cv = kf,
        scoring = 'neg_mean_squared_error'
    )
    
    val_rmse = result['test_score'].mean()
    
    return val_rmse

In [12]:
# optuna を実行
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_tirals=100)

[I 2022-10-31 17:34:10,075] A new study created in memory with name: no-name-d5305729-4c3e-4037-ad1c-d1fb490d7729


NameError: name 'objective' is not defined